In [1]:
!pwd

/aiffel/aiffel/Aimmo


In [2]:
%cd mmdetection

#완디비 설치
#!pip install -q --upgrade wandb
#!pip3 install openmim
#!mim install mmcv-full
#!pip install -e .

/aiffel/aiffel/Aimmo/mmdetection


In [3]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.6


In [4]:
# # 완디비 로그인
wandb.login(key= '')

wandb: Currently logged in as: cychuckpark (yolo_x). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [5]:
wandb.init(project="test-project", entity="yolo_x")

In [6]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'bus', 'truck','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_txt')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [7]:
config_file = 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg = Config.fromfile(config_file)

cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'


cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'


cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/'
cfg.data.test.ann_file = 'middle_test.txt'
cfg.data.test.img_prefix = 'image'

cfg.model.roi_head.bbox_head.num_classes = 4

cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg.work_dir = 'middle'

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'

cfg.log_config.interval = 100
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
#cfg.evaluation.save_best='auto'
cfg.data.train.pipeline[2].img_scale=[(960,512)]#['Resize']
cfg.train_pipeline[2].img_scale=[(960,512)]
cfg.test_pipeline[1].img_scale=[(960,512)]
cfg.data.val.pipeline[1].img_scale = [(960,512)]
cfg.data.test.pipeline[1].img_scale = [(960,512)]
cfg.runner.max_epochs = 20

cfg.data.samples_per_gpu = 14 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 0 # Worker to pre-fetch data for each single GPU
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [8]:
cfg.model.rpn_head.anchor_generator

{'type': 'AnchorGenerator',
 'scales': [8],
 'ratios': [0.5, 1.0, 2.0],
 'strides': [4, 8, 16, 32, 64]}

In [9]:
cfg.model.rpn_head.anchor_generator['scales'] = [4]
cfg.model.rpn_head.anchor_generator['strides'] = [2,4,8,16,32]

In [10]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'test-project', 'entity' : 'yolo_x', 'name' : 'middle'})] 

In [11]:
[build_dataset(cfg.data.val)]

/aiffel/aiffel/Aimmo/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


[
 Aimmo Train dataset with number of images 2323, and instance counts: 
 +----------+-------+----------+-------+-----------+-------+----------------+-------+---------------+-------+
 | category | count | category | count | category  | count | category       | count | category      | count |
 +----------+-------+----------+-------+-----------+-------+----------------+-------+---------------+-------+
 | 0 [car]  | 13124 | 1 [bus]  | 1786  | 2 [truck] | 3084  | 3 [pedestrian] | 3063  | -1 background | 0     |
 +----------+-------+----------+-------+-----------+-------+----------------+-------+---------------+-------+]

In [ ]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-12-07 13:59:17,404 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-07 13:59:17,591 - mmdet - INFO - load checkpoint from local path: checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2022-12-07 13:59:19,030 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([16, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([16]).
20

2022-12-07 14:04:43,473 - mmdet - INFO - Epoch [1][100/1494]	lr: 3.976e-03, eta: 1 day, 2:18:18, time: 3.180, data_time: 1.595, memory: 11276, loss_rpn_cls: 0.0235, loss_rpn_bbox: 0.0010, loss_cls: 0.1265, acc: 97.3119, loss_bbox: 0.0087, loss: 0.1598
2022-12-07 14:10:09,853 - mmdet - INFO - Epoch [1][200/1494]	lr: 7.972e-03, eta: 1 day, 2:33:45, time: 3.264, data_time: 1.599, memory: 11276, loss_rpn_cls: 0.0064, loss_rpn_bbox: 0.0009, loss_cls: 0.0227, acc: 99.2139, loss_bbox: 0.0064, loss: 0.0364
2022-12-07 14:15:36,555 - mmdet - INFO - Epoch [1][300/1494]	lr: 1.197e-02, eta: 1 day, 2:35:48, time: 3.267, data_time: 1.613, memory: 11276, loss_rpn_cls: 0.0034, loss_rpn_bbox: 0.0009, loss_cls: 0.0196, acc: 99.3344, loss_bbox: 0.0072, loss: 0.0312
2022-12-07 14:21:03,638 - mmdet - INFO - Epoch [1][400/1494]	lr: 1.596e-02, eta: 1 day, 2:34:34, time: 3.271, data_time: 1.618, memory: 11276, loss_rpn_cls: 0.0031, loss_rpn_bbox: 0.0009, loss_cls: 0.0149, acc: 99.5188, loss_bbox: 0.0076, loss:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.0 task/s, elapsed: 460s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-07 15:28:51,595 - mmdet - INFO - 
+------------+-------+------+--------+-------+
| class      | gts   | dets | recall | ap    |
+------------+-------+------+--------+-------+
| car        | 13124 | 1324 | 0.019  | 0.011 |
| bus        | 1786  | 296  | 0.023  | 0.013 |
| truck      | 3084  | 326  | 0.008  | 0.006 |
| pedestrian | 3063  | 793  | 0.012  | 0.005 |
+------------+-------+------+--------+-------+
| mAP        |       |      |        | 0.009 |
+------------+-------+------+--------+-------+
2022-12-07 15:28:51,695 - mmdet - INFO - Epoch(val) [1][2323]	AP50: 0.0090, mAP: 0.0086
2022-12-07 15:34:26,939 - mmdet - INFO - Epoch [2][100/1494]	lr: 2.000e-02, eta: 1 day, 0:15:55, time: 3.346, data_time: 1.637, memory: 11276, loss_rpn_cls: 0.0016, loss_rpn_bbox: 0.0008, loss_cls: 0.0068, acc: 99.7737, loss_bbox: 0.0050, loss: 0.0142
2022-12-07 15:39:59,557 - mmdet - INFO - Epoch [2][200/1494]	lr: 2.000e-02, eta: 1 day, 0:17:22, time: 3.326, data_time: 1.638, memory: 11276, loss_

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.0 task/s, elapsed: 464s, ETA:     0s
---------------iou_thr: 0.5---------------


...

2022-12-08 06:26:41,817 - mmdet - INFO - 
+------------+-------+------+--------+-------+
| class      | gts   | dets | recall | ap    |
+------------+-------+------+--------+-------+
| car        | 13124 | 877  | 0.023  | 0.020 |
| bus        | 1786  | 231  | 0.029  | 0.022 |
| truck      | 3084  | 282  | 0.012  | 0.009 |
| pedestrian | 3063  | 560  | 0.020  | 0.012 |
+------------+-------+------+--------+-------+
| mAP        |       |      |        | 0.016 |
+------------+-------+------+--------+-------+
2022-12-08 06:26:41,908 - mmdet - INFO - Epoch(val) [11][2323]	AP50: 0.0160, mAP: 0.0156
2022-12-08 06:32:06,862 - mmdet - INFO - Epoch [12][100/1494]	lr: 2.000e-04, eta: 11:27:01, time: 3.243, data_time: 1.570, memory: 11276, loss_rpn_cls: 0.0013, loss_rpn_bbox: 0.0007, loss_cls: 0.0037, acc: 99.8753, loss_bbox: 0.0027, loss: 0.0084
2022-12-08 06:37:29,366 - mmdet - INFO - Epoch [12][200/1494]	lr: 2.000e-04, eta: 11:22:03, time: 3.225, data_time: 1.554, memory: 11276, loss_rpn_cls: 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.3 task/s, elapsed: 441s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-08 07:54:42,305 - mmdet - INFO - 
+------------+-------+------+--------+-------+
| class      | gts   | dets | recall | ap    |
+------------+-------+------+--------+-------+
| car        | 13124 | 916  | 0.023  | 0.020 |
| bus        | 1786  | 238  | 0.029  | 0.021 |
| truck      | 3084  | 283  | 0.012  | 0.009 |
| pedestrian | 3063  | 537  | 0.020  | 0.012 |
+------------+-------+------+--------+-------+
| mAP        |       |      |        | 0.015 |
+------------+-------+------+--------+-------+
2022-12-08 07:54:42,398 - mmdet - INFO - Epoch(val) [12][2323]	AP50: 0.0150, mAP: 0.0154
2022-12-08 08:00:08,478 - mmdet - INFO - Epoch [13][100/1494]	lr: 2.000e-04, eta: 10:09:12, time: 3.254, data_time: 1.583, memory: 11276, loss_rpn_cls: 0.0015, loss_rpn_bbox: 0.0007, loss_cls: 0.0032, acc: 99.8908, loss_bbox: 0.0023, loss: 0.0077
2022-12-08 08:05:32,340 - mmdet - INFO - Epoch [13][200/1494]	lr: 2.000e-04, eta: 10:04:14, time: 3.239, data_time: 1.571, memory: 11276, loss_rpn_cls: 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.2 task/s, elapsed: 445s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-08 09:22:59,496 - mmdet - INFO - 
+------------+-------+------+--------+-------+
| class      | gts   | dets | recall | ap    |
+------------+-------+------+--------+-------+
| car        | 13124 | 958  | 0.023  | 0.020 |
| bus        | 1786  | 238  | 0.029  | 0.021 |
| truck      | 3084  | 271  | 0.012  | 0.009 |
| pedestrian | 3063  | 541  | 0.020  | 0.013 |
+------------+-------+------+--------+-------+
| mAP        |       |      |        | 0.015 |
+------------+-------+------+--------+-------+
2022-12-08 09:22:59,589 - mmdet - INFO - Epoch(val) [13][2323]	AP50: 0.0150, mAP: 0.0154
2022-12-08 09:28:25,823 - mmdet - INFO - Epoch [14][100/1494]	lr: 2.000e-04, eta: 8:51:50, time: 3.256, data_time: 1.585, memory: 11276, loss_rpn_cls: 0.0014, loss_rpn_bbox: 0.0007, loss_cls: 0.0036, acc: 99.8779, loss_bbox: 0.0028, loss: 0.0085
2022-12-08 09:33:50,576 - mmdet - INFO - Epoch [14][200/1494]	lr: 2.000e-04, eta: 8:46:51, time: 3.247, data_time: 1.576, memory: 11276, loss_rpn_cls: 0.